In [1]:
import argparse
import cv2
import numpy as np

In [12]:
img = cv2.imread('test1.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray,128,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

In [13]:
cv2.imshow('thresh',thresh)
cv2.waitKey(0)

-1

In [14]:
output = cv2.connectedComponentsWithStats(thresh,4,cv2.CV_32S)
(numLabels, labels, stats, centroids) = output

In [15]:
# loop over the number of unique connected component labels
for i in range(0, numLabels):
    # if this is the first component then we examine the
    # *background* (typically we would just ignore this
    # component in our loop)
    if i == 0:
        text = "examining component {}/{} (background)".format(i + 1, numLabels)
    # otherwise, we are examining an actual connected component
    else:
        text = "examining component {}/{}".format( i + 1, numLabels)
    # print a status message update for the current connected
    # component
    print("[INFO] {}".format(text))
    # extract the connected component statistics and centroid for
    # the current label
    x = stats[i, cv2.CC_STAT_LEFT]
    y = stats[i, cv2.CC_STAT_TOP]
    w = stats[i, cv2.CC_STAT_WIDTH]
    h = stats[i, cv2.CC_STAT_HEIGHT]
    area = stats[i, cv2.CC_STAT_AREA]
    (cX, cY) = centroids[i]

[INFO] examining component 1/24 (background)
[INFO] examining component 2/24
[INFO] examining component 3/24
[INFO] examining component 4/24
[INFO] examining component 5/24
[INFO] examining component 6/24
[INFO] examining component 7/24
[INFO] examining component 8/24
[INFO] examining component 9/24
[INFO] examining component 10/24
[INFO] examining component 11/24
[INFO] examining component 12/24
[INFO] examining component 13/24
[INFO] examining component 14/24
[INFO] examining component 15/24
[INFO] examining component 16/24
[INFO] examining component 17/24
[INFO] examining component 18/24
[INFO] examining component 19/24
[INFO] examining component 20/24
[INFO] examining component 21/24
[INFO] examining component 22/24
[INFO] examining component 23/24
[INFO] examining component 24/24


In [16]:
output = img.copy()
cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 3)
cv2.circle(output, (int(cX), int(cY)), 4, (0, 0, 255), -1)

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [17]:
# construct a mask for the current connected component by
# finding a pixels in the labels array that have the current
# connected component ID
componentMask = (labels == i).astype("uint8") * 255
# show our output image and connected component mask
cv2.imshow("Output", output)
cv2.imshow("Connected Component", componentMask)
cv2.waitKey(0)

-1

In [18]:
import numpy as np

In [19]:
# load the input image from disk, convert it to grayscale, and
# threshold it
# img = cv2.imread(args["image"])
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
# apply connected component analysis to the thresholded image
# output = cv2.connectedComponentsWithStats(thresh, args["connectivity"], cv2.CV_32S)
# (numLabels, labels, stats, centroids) = output
# initialize an output mask to store all characters parsed from
# the license plate

mask = np.zeros(gray.shape, dtype="uint8")

In [20]:
# loop over the number of unique connected component labels, skipping
# over the first label (as label zero is the background)
for i in range(1, numLabels):
    # extract the connected component statistics for the current
    # label
    x = stats[i, cv2.CC_STAT_LEFT]
    y = stats[i, cv2.CC_STAT_TOP]
    w = stats[i, cv2.CC_STAT_WIDTH]
    h = stats[i, cv2.CC_STAT_HEIGHT]
    area = stats[i, cv2.CC_STAT_AREA]
    
    # ensure the width, height, and area are all neither too small
    # nor too big
    keepWidth = w > 10 and w < 500
    keepHeight = h > 70 and h < 500
    keepArea = area > 100 and area < 15000
    # ensure the connected component we are examining passes all
    # three tests
    if all((keepWidth, keepHeight, keepArea)):
        # construct a mask for the current connected component and
        # then take the bitwise OR with the mask
        print("[INFO] keeping connected component '{}'".format(i))
        componentMask = (labels == i).astype("uint8") * 255
        mask = cv2.bitwise_or(mask, componentMask)

[INFO] keeping connected component '2'
[INFO] keeping connected component '3'
[INFO] keeping connected component '4'
[INFO] keeping connected component '5'
[INFO] keeping connected component '6'
[INFO] keeping connected component '7'
[INFO] keeping connected component '8'
[INFO] keeping connected component '9'
[INFO] keeping connected component '10'


In [21]:
# show the original input image and the mask for the license plate
# characters
cv2.imshow("Image", img)
cv2.imshow("Characters", mask)
cv2.waitKey(0)
cv2.destroyAllWindows()